In [83]:
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage
import requests
from dotenv import load_dotenv
import os

In [84]:
load_dotenv()

True

In [85]:
# tool create
from langchain_core.tools import InjectedToolArg
from typing import Annotated

API_KEY = os.getenv("EXCHANGE_API_KEY")

@tool
def get_conversion_factor(base_currency: str, target_currency: str) -> float:
  """
  This function fetches the currency conversion factor between a given base currency and a target currency
  """
  url = f'https://v6.exchangerate-api.com/v6/{API_KEY}/pair/{base_currency}/{target_currency}'

  response = requests.get(url)

  return response.json()

@tool
def convert(base_currency_value: int, conversion_rate: Annotated[float, InjectedToolArg]) -> float:
  """
  given a currency conversion rate this function calculates the target currency value from a given base currency value
  """

  return base_currency_value * conversion_rate


In [86]:
convert.args

{'base_currency_value': {'title': 'Base Currency Value', 'type': 'integer'},
 'conversion_rate': {'title': 'Conversion Rate', 'type': 'number'}}

In [87]:
get_conversion_factor.invoke({'base_currency':'USD','target_currency':'BDT'})

{'result': 'success',
 'documentation': 'https://www.exchangerate-api.com/docs',
 'terms_of_use': 'https://www.exchangerate-api.com/terms',
 'time_last_update_unix': 1756252801,
 'time_last_update_utc': 'Wed, 27 Aug 2025 00:00:01 +0000',
 'time_next_update_unix': 1756339201,
 'time_next_update_utc': 'Thu, 28 Aug 2025 00:00:01 +0000',
 'base_code': 'USD',
 'target_code': 'BDT',
 'conversion_rate': 121.8849}

In [88]:
convert.invoke({'base_currency_value':10, 'conversion_rate':121.8849})

1218.849

In [89]:
# tool binding
llm = ChatOpenAI()

In [90]:
llm_with_tools = llm.bind_tools([get_conversion_factor, convert])

In [141]:
# Tool Calling
messages = [HumanMessage('What is the conversion factor between USD and BDT, and based on that can you convert 10 USD to BDT')]

In [142]:
messages

[HumanMessage(content='What is the conversion factor between USD and BDT, and based on that can you convert 10 USD to BDT', additional_kwargs={}, response_metadata={})]

In [143]:
ai_message = llm_with_tools.invoke(messages)

In [144]:
messages.append(ai_message)

In [145]:
ai_message

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_5Qh4q8euqhKQByLyBX8mTJEw', 'function': {'arguments': '{"base_currency": "USD", "target_currency": "BDT"}', 'name': 'get_conversion_factor'}, 'type': 'function'}, {'id': 'call_NLuRIdMUIeiGK4IOqPhK0hQq', 'function': {'arguments': '{"base_currency_value": 10}', 'name': 'convert'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 52, 'prompt_tokens': 122, 'total_tokens': 174, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-C97zET7bQ8Xpv6rKZghrXyUhfLpz1', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--162c876c-c9e2-4003-8e8a-0ad7a165c2a9-0', tool_calls=[{'name': 'get_conversion_factor', 'args': {'base_cur

In [146]:
ai_message.tool_calls

[{'name': 'get_conversion_factor',
  'args': {'base_currency': 'USD', 'target_currency': 'BDT'},
  'id': 'call_5Qh4q8euqhKQByLyBX8mTJEw',
  'type': 'tool_call'},
 {'name': 'convert',
  'args': {'base_currency_value': 10},
  'id': 'call_NLuRIdMUIeiGK4IOqPhK0hQq',
  'type': 'tool_call'}]

In [147]:
import json

for tool_call in ai_message.tool_calls:
  # execute the 1st tool and get the value of conversion rate
  if tool_call['name'] == 'get_conversion_factor':
    tool_message1 = get_conversion_factor.invoke(tool_call)
    # fetch this conversion rate
    conversion_rate = json.loads(tool_message1.content)['conversion_rate']
    # append this tool message to messages list
    messages.append(tool_message1)
  # execute the 2nd tool using the conversion rate from tool 1
  if tool_call['name'] == 'convert':
    # fetch the current arg
    tool_call['args']['conversion_rate'] = conversion_rate
    tool_message2 = convert.invoke(tool_call)
    messages.append(tool_message2)

In [148]:
messages

[HumanMessage(content='What is the conversion factor between USD and BDT, and based on that can you convert 10 USD to BDT', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_5Qh4q8euqhKQByLyBX8mTJEw', 'function': {'arguments': '{"base_currency": "USD", "target_currency": "BDT"}', 'name': 'get_conversion_factor'}, 'type': 'function'}, {'id': 'call_NLuRIdMUIeiGK4IOqPhK0hQq', 'function': {'arguments': '{"base_currency_value": 10}', 'name': 'convert'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 52, 'prompt_tokens': 122, 'total_tokens': 174, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-C97zET7bQ8Xpv6rKZghrXyUhfLpz1', 'service_tier': 'defa

In [149]:
llm_with_tools.invoke(messages)

AIMessage(content='The conversion factor between USD and BDT is 121.8849.\n\nBased on this conversion factor, 10 USD is equal to 1218.849 BDT.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 36, 'prompt_tokens': 334, 'total_tokens': 370, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-C97zGG7MABCiLXGHevIakoVIjCuXP', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--5a0af86e-87ae-46b7-a2ec-248c9d19b0ee-0', usage_metadata={'input_tokens': 334, 'output_tokens': 36, 'total_tokens': 370, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [150]:
llm_with_tools.invoke(messages).content

'The conversion factor between USD and BDT is 121.8849.\nBased on this conversion factor, 10 USD is equivalent to 1218.849 BDT.'